In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import pandas as pd
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import folium
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from numpy.linalg import norm

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nachi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nachi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# TASK 1

In [26]:
# Businesses data
df_businesses = pd.read_csv("C:/Users/nachi/Desktop/Info_Retrieval/Final_Project/archive/businesses.csv", sep='\t', encoding = 'ISO-8859-1')
print (len(df_businesses))
df_businesses.head(10)

100000


,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",NaN
1,1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
5,5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
6,6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
7,7,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,FL,33707,27.766590,-82.732983,3.5,5,1,NaN,"Synagogues, Religious Organizations","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
8,8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...","Pubs, Restaurants, Italian, Bars, American (Tr...",NaN
9,9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."


In [27]:
df_businesses.tail()

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
99995,99995,XwEzff2lxcn2MoWdRaMqUg,The Groom Room,210 Louise Ave,Nashville,TN,37203,36.151926,-86.804949,4.5,23,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Waxing, Spray Tanning, Permanent Makeup, Eyela...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-19:0', ..."
99996,99996,BjRz_MdoqUn6FYDsjAqn6Q,61Five Health & Wellness,1913 Church St,Nashville,TN,37203,36.153587,-86.799671,5.0,78,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","IV Hydration, Beauty & Spas, Medical Centers, ...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
99997,99997,2pWRW7ASh_Bw5q5V64Luuw,Ridge Carlton Apartments,7373 Ridge Ave,Philadelphia,PA,19128,40.047768,-75.234005,2.5,9,1,{'BusinessAcceptsCreditCards': 'True'},"Real Estate, Home Services, Apartments, Real E...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
99998,99998,ccwhUs2fCLWTQLMa1L_xNQ,China Buffet,57 Fenton Plz,Fenton,MO,63026,38.511352,-90.442368,2.0,10,0,"{'RestaurantsAttire': ""u'casual'"", 'GoodForKid...","Chinese, Restaurants",NaN
99999,99999,6bhEQ4Y3qAJuMxy1xpdJnA,Duncan Family Healthcare,"6630 S Mccarran Blvd, Ste 12A",Reno,NV,89509,39.476217,-119.801622,3.0,42,1,"{'AcceptsInsurance': 'True', 'WiFi': ""u'no'"", ...","Health & Medical, Weight Loss Centers, Family ...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '..."


In [28]:
df_businesses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100000 non-null  int64  
 1   business_id   100000 non-null  object 
 2   name          100000 non-null  object 
 3   address       96606 non-null   object 
 4   city          100000 non-null  object 
 5   state         100000 non-null  object 
 6   postal_code   99958 non-null   object 
 7   latitude      100000 non-null  float64
 8   longitude     100000 non-null  float64
 9   stars         100000 non-null  float64
 10  review_count  100000 non-null  int64  
 11  is_open       100000 non-null  int64  
 12  attributes    90915 non-null   object 
 13  categories    99931 non-null   object 
 14  hours         84564 non-null   object 
dtypes: float64(3), int64(3), object(9)
memory usage: 11.4+ MB


In [29]:
# Count businesses in each city
city_counts = df_businesses['city'].value_counts()

# Display the count of businesses in each city
print("Count of businesses in each city:")
city_counts.head(15)


Count of businesses in each city:


Philadelphia        9670
Tucson              6166
Tampa               6030
Indianapolis        5003
Nashville           4606
New Orleans         4138
Reno                3968
Edmonton            3356
Saint Louis         3268
Santa Barbara       2539
Boise               1958
Clearwater          1473
Metairie            1101
Saint Petersburg    1094
Sparks              1085
Name: city, dtype: int64

In [30]:
# Filter the DataFrame for businesses in Las Vegas
businesses_in_las_vegas = df_businesses[df_businesses['city'] == 'Las Vegas']

# Count the number of businesses in Las Vegas
count_las_vegas = len(businesses_in_las_vegas)

# Display the count
print("Count of businesses in Las Vegas:", count_las_vegas)


Count of businesses in Las Vegas: 0


In [31]:
# Filter the DataFrame for businesses in Las Vegas
businesses_in_las_vegas = df_businesses[df_businesses['city'] == 'Toronto']

# Count the number of businesses in Las Vegas
count_las_vegas = len(businesses_in_las_vegas)

# Display the count
print("Count of businesses in Toronto:", count_las_vegas)


Count of businesses in Toronto: 0


In [32]:
# Filter the DataFrame for businesses in Las Vegas
businesses_in_las_vegas = df_businesses[df_businesses['city'] == 'Chicago']

# Count the number of businesses in Las Vegas
count_las_vegas = len(businesses_in_las_vegas)

# Display the count
print("Count of businesses in Chicago:", count_las_vegas)


Count of businesses in Chicago: 1


In [33]:
# Filter the DataFrame for businesses in Las Vegas
businesses_in_las_vegas = df_businesses[df_businesses['city'] == 'Atlanta']

# Count the number of businesses in Las Vegas
count_las_vegas = len(businesses_in_las_vegas)

# Display the count
print("Count of businesses in Toronto:", count_las_vegas)

Count of businesses in Toronto: 0


In [34]:
# 1. Best restaurants in Nashville
query_1 = df_businesses[(df_businesses['city'] == 'Nashville') & (df_businesses['categories'].str.contains('Restaurant'))]
query_1['categories'] = query_1['categories'].astype(str)
query_1 = query_1.sort_values(by=['stars', 'review_count', 'name'], ascending=[False, False, True]).head(5)
print("Best restaurants in Nashville:")
print(query_1[['stars', 'name', 'review_count']])

# 2. Best Chinese restaurants in Tampa
query_2 = df_businesses[(df_businesses['city'] == 'Tampa') & (df_businesses['categories'].str.contains('Chinese'))]
query_2['categories'] = query_2['categories'].astype(str)
query_2 = query_2.sort_values(by=['stars', 'review_count', 'name'], ascending=[False, False, True]).head(5)
print("\nBest Chinese restaurants in Tampa:")
print(query_2[['stars', 'name', 'review_count']])

# 3. Pubs in Philadelphia that are Wheelchair Accessible
query_3 = df_businesses[(df_businesses['city'] == 'Philadelphia') & (df_businesses['categories'].str.contains('Pubs')) & (df_businesses['attributes'].str.contains('WheelchairAccessible'))]
query_3['categories'] = query_3['categories'].astype(str)
query_3 = query_3.sort_values(by=['stars', 'review_count', 'name'], ascending=[False, False, True]).head(5)
print("\nPubs in Philadelphia that are Wheelchair Accessible:")
print(query_3[['stars', 'name', 'review_count']])


Best restaurants in Nashville:
       stars                                       name  review_count
95158    5.0                              Big Al's Deli           390
31772    5.0                        Tasty And Delicious           260
51628    5.0                    D'Andrews Bakery & Cafe           212
73120    5.0  Edessa Restaurant Kurdish Turkish Cuisine           198
37626    5.0               The CafÃÂ© at Thistle Farms           189

Best Chinese restaurants in Tampa:
       stars                     name  review_count
56096    5.0             Fuzion Spice            80
61094    5.0  Pure Zen Mobile Massage             8
53908    5.0       Luxe Beauty Studio             7
85221    5.0              Anna Health             6
34261    4.5            Soul Of Korea           238

Pubs in Philadelphia that are Wheelchair Accessible:
       stars                     name  review_count
94105    4.5      Lucky's Last Chance           579
17602    4.5               SouthHouse      

C:\Users\nachi\AppData\Local\Temp\ipykernel_18188\2919932104.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_1['categories'] = query_1['categories'].astype(str)
C:\Users\nachi\AppData\Local\Temp\ipykernel_18188\2919932104.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_2['categories'] = query_2['categories'].astype(str)
C:\Users\nachi\AppData\Local\Temp\ipykernel_18188\2919932104.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [35]:
# 4. Business hours for "DeSandro on Main" in Philadelphia for Friday
query_4 = df_businesses[(df_businesses['name'] == 'DeSandro on Main') & (df_businesses['city'] == 'Philadelphia')]
hours_friday_str = query_4['hours'].values[0]
hours_friday_dict = ast.literal_eval(hours_friday_str)
hours_friday = hours_friday_dict.get('Friday', 'Hours not available')
print("\nBusiness hours for 'DeSandro on Main' in Philadelphia for Friday:")
print(hours_friday)


Business hours for 'DeSandro on Main' in Philadelphia for Friday:
17:0-0:30


In [36]:
# Function to geocode the city name and return its coordinates
def geocode_city(city_name):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(city_name)
    return (location.latitude, location.longitude)

# Enter the city name
city_name = "Tampa"

# Geocode the city name to get its coordinates
city_coords = geocode_city(city_name)

# Define the coordinates of the landmark (example coordinates)
landmark_coords = city_coords

# Calculate distances between landmark and each pub
df_businesses['distance_to_landmark'] = df_businesses.apply(lambda row: geodesic(landmark_coords, (row['latitude'], row['longitude'])).miles, axis=1)

# Filter pubs near the landmark (e.g., within 5 miles)
nearby_pubs = df_businesses[(df_businesses['categories'].str.contains('Pubs')) & (df_businesses['distance_to_landmark'] <= 5)]

# Select top 5 closest pubs
top_5_pubs = nearby_pubs.nsmallest(10, 'distance_to_landmark')

# Plot pubs on a map
map_center = landmark_coords  # Set the center of the map to the landmark coordinates
pubs_map = folium.Map(location=map_center, zoom_start=12,tiles="CartoDB dark_matter")

# Add markers for each pub
for index, row in top_5_pubs.iterrows():
    folium.Marker(location=(row['latitude'], row['longitude']), popup=row['name']).add_to(pubs_map)

# Save the map as an HTML file or display it
# pubs_map.save("pubs_near_landmark_map.html")
pubs_map


In [37]:
# Enter the city name
city_name = "Indianapolis"

# Geocode the city name to get its coordinates
city_coords = geocode_city(city_name)

# Define the coordinates of the landmark (example coordinates)
landmark_coords = city_coords

# Calculate distances between landmark and each pub
df_businesses['distance_to_landmark'] = df_businesses.apply(lambda row: geodesic(landmark_coords, (row['latitude'], row['longitude'])).miles, axis=1)

# Filter pubs near the landmark (e.g., within 5 miles)
nearby_pubs = df_businesses[(df_businesses['categories'].str.contains('Pubs')) & (df_businesses['distance_to_landmark'] <= 5)]

# Select top 5 closest pubs
top_5_pubs = nearby_pubs.nsmallest(10, 'distance_to_landmark')

# Plot pubs on a map
map_center = landmark_coords  # Set the center of the map to the landmark coordinates
pubs_map = folium.Map(location=map_center, zoom_start=12,tiles="CartoDB dark_matter")

# Add markers for each pub with the business name as a popup
for index, row in top_5_pubs.iterrows():
    folium.Marker(location=(row['latitude'], row['longitude']), popup=row['name']).add_to(pubs_map)

# Save the map as an HTML file or display it
# pubs_map.save("pubs_near_landmark_map.html")
pubs_map


# TASK 2

In [39]:
# Load GloVe embeddings
def load_glove(vector_len):
    #"C:/Users/nachi/Desktop/Info_Retrieval/Final_Project/archive/glove.6B.50d.txt"
    GloVe = f"C:/Users/nachi/Desktop/Info_Retrieval/Final_Project/archive/glove.6B.{vector_len}d.txt"
    
    embeddings_dict = {} # embeddings of all words stored in a dictionary

    with open(GloVe, 'r', encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = str(values[0])

            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
    return embeddings_dict

def tokenize_text(text):
    stop_words = set(stopwords.words('english'))
    token_list = []
    text_str = re.sub(r'[^\w\s]', '', text.lower()) # drop non-alphanumeric or non-white-space
    for word in word_tokenize(text_str):
        if word not in stop_words and len(word) > 1 and len(word) < 20:
            token_list.append(word)
    return token_list

def create_embedding(text, embeddings_dict):
    tokens = tokenize_text(text)
    embedding_dim = len(next(iter(embeddings_dict.values())))
    embedding = np.zeros(embedding_dim)
    count = 0
    for token in tokens:
        if token in embeddings_dict:
            embedding += embeddings_dict[token]
            count += 1
    if count != 0:
        embedding /= count
    return embedding


In [40]:
# Step 1: Creating embeddings for users and businesses

# Load training data
df_train = pd.read_csv("C:/Users/nachi/Desktop/Info_Retrieval/Final_Project/archive/reviews_train.csv", sep='\t', encoding = 'ISO-8859-1')

# Load GloVe embeddings
embeddings_dict = load_glove(50)  # Assuming you want embeddings of size 50
# Print number of words in embeddings
print("Number of words in embeddings:", len(embeddings_dict))

# Create user embeddings
user_embeddings = {}
for user_id, group in df_train.groupby('user_id'):
    combined_text = ' '.join(group['text'])
    user_embedding = create_embedding(combined_text, embeddings_dict)
    user_embeddings[user_id] = user_embedding

# Print number of users
print("Number of users:", len(user_embeddings))

# Create business embeddings
business_embeddings = {}
for business_id, group in df_train.groupby('business_id'):
    combined_text = ' '.join(group['text'])
    business_embedding = create_embedding(combined_text, embeddings_dict)
    business_embeddings[business_id] = business_embedding

# Print number of businesses
print("Number of businesses:", len(business_embeddings))

Number of words in embeddings: 400000
Number of users: 65400
Number of businesses: 9406


In [41]:
# Print few rows of user embeddings
print("User Embeddings:")
for user_id, embedding in list(user_embeddings.items())[:5]:
    print(f"User ID: {user_id}, Embedding: {embedding}")

User Embeddings:
User ID: ---2PmXbF47D870stH1jqA, Embedding: [ 0.18179334  0.15679728  0.02431584 -0.12751953  0.26508306 -0.12956303
 -0.49503709 -0.04984553  0.07932762  0.02094368 -0.20207122 -0.02350481
  0.09784356 -0.02976277  0.69897515 -0.03234853 -0.25634837 -0.07686694
 -0.30679888 -0.40913306  0.2562667   0.06959725  0.4252276   0.18225016
  0.06964164 -0.90263937 -0.13010307 -0.09008684  0.19131494 -0.10242181
  2.41408094  0.39486203 -0.19766426  0.05249237  0.14416981 -0.05482515
  0.19747244  0.18232825 -0.00312194 -0.08588853 -0.01956597  0.12822209
  0.01769935  0.26543982  0.01164022  0.1920214  -0.09024576  0.15077397
 -0.07783591  0.26284768]
User ID: --4AjktZiHowEIBCMd4CZA, Embedding: [-3.58494818e-02  6.20257128e-02 -3.37085512e-01 -2.95290001e-01
  2.49831226e-01 -6.68525830e-02 -2.87836011e-01  4.42720546e-02
  9.83580166e-04  3.62191029e-01 -2.33753244e-01  1.09390131e-01
  1.50685776e-01  1.34174337e-01  2.14717573e-01  1.99534062e-01
  1.85975709e-01  2.01370

In [42]:
# Print few rows of business embeddings
print("Business Embeddings:")
for business_id, embedding in list(business_embeddings.items())[:5]:
    print(f"Business ID: {business_id}, Embedding: {embedding}")

Business Embeddings:
Business ID: --ZVrH2X2QXBFdCilbirsw, Embedding: [ 0.19472704 -0.02487518 -0.11295627 -0.20675501  0.34828381 -0.01626434
 -0.41368887  0.04808977 -0.14528724  0.09125711 -0.13698143  0.07080108
 -0.05573262  0.08072954  0.31671834  0.0848695   0.03829465  0.0664698
 -0.12431379 -0.46393942  0.07091058  0.12803239  0.11365564  0.22811588
  0.19575421 -1.05205423 -0.42504606  0.16484893  0.44755416 -0.26156719
  2.31517996  0.29599055 -0.05742758  0.01360496  0.07636354  0.06868283
 -0.01411432  0.27382766  0.03864675 -0.25205649 -0.00710337  0.06289474
  0.04954738  0.15911209  0.12344547  0.22281536 -0.08475792 -0.04540261
  0.0660291   0.16990117]
Business ID: --_9CAxgfXZmoFdNIRrhHA, Embedding: [ 1.72894736e-01  1.85782174e-02  1.25514593e-01 -1.92676009e-01
  3.32435467e-01 -1.26160386e-01 -4.50941228e-01  6.10671977e-02
 -8.58262461e-02 -4.32355178e-02 -1.28198046e-01 -1.05390943e-02
 -1.56905623e-01  1.93341201e-02  3.83411534e-01  1.95059584e-01
  1.25032234e-

In [43]:
# Step 2: Predicting star ratings for user-business pairs in the test dataset

# Load test data
df_test = pd.read_csv("C:/Users/nachi/Desktop/Info_Retrieval/Final_Project/archive/reviews_test_all.csv", sep='\t', encoding='ISO-8859-1')

# Prediction and evaluation
total_squared_error = 0
for _, row in df_test.iterrows():
    user_id = row['user_id']
    business_id = row['business_id']
    stars = row['stars']
    
    if user_id in user_embeddings and business_id in business_embeddings:
        user_embedding = user_embeddings[user_id]
        business_embedding = business_embeddings[business_id]
        predicted_stars = np.dot(user_embedding, business_embedding)
        total_squared_error += ((predicted_stars - stars) ** 2)

# Calculate RMSE
mean_squared_error = total_squared_error / len(df_test)
root_mean_squared_error = np.sqrt(mean_squared_error)
print("Root Mean Squared Error:", root_mean_squared_error)

Root Mean Squared Error: 2.436257743155378


# LINEAR REGRESSION MODEL

In [44]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Prepare features and target variable for training
X_train = []
y_train = []
for _, row in df_train.iterrows():
    user_id = row['user_id']
    business_id = row['business_id']
    stars = row['stars']
    
    if user_id in user_embeddings and business_id in business_embeddings:
        user_embedding = user_embeddings[user_id]
        business_embedding = business_embeddings[business_id]
        X_train.append(list(user_embedding) + list(business_embedding))
        y_train.append(stars)

# Prepare features and target variable for testing
X_test = []
y_test = []
for _, row in df_test.iterrows():
    user_id = row['user_id']
    business_id = row['business_id']
    stars = row['stars']
    
    if user_id in user_embeddings and business_id in business_embeddings:
        user_embedding = user_embeddings[user_id]
        business_embedding = business_embeddings[business_id]
        X_test.append(list(user_embedding) + list(business_embedding))
        y_test.append(stars)

# Initialize and fit linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict star ratings
y_pred = model.predict(X_test)

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error:", rmse)

# Evaluate accuracy on the training dataset
y_train_pred = model.predict(X_train)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
print("Training Root Mean Squared Error:", train_rmse)

Root Mean Squared Error: 1.2048166983021553
Training Root Mean Squared Error: 1.0856754612360127


# XGBoost Regressor

In [22]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Initialize and fit XGBoost regressor model
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)

# Predict star ratings
y_pred_xgb = xgb_model.predict(X_test)

# Calculate RMSE for XGBoost model
rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
print("Root Mean Squared Error (XGBoost):", rmse_xgb)

# Evaluate accuracy on the training dataset
y_train_pred_xgb = xgb_model.predict(X_train)
train_rmse_xgb = mean_squared_error(y_train, y_train_pred_xgb, squared=False)
print("Training Root Mean Squared Error (XGBoost):", train_rmse_xgb)

Root Mean Squared Error (XGBoost): 1.2173186064313442
Training Root Mean Squared Error (XGBoost): 0.8173946334663076


In [115]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Define hyperparameters grid for tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Initialize XGBoost regressor model
xgb_model = XGBRegressor()

# Initialize GridSearchCV with the XGBoost model and hyperparameters grid
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_root_mean_squared_error')

# Perform grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Get the best hyperparameters found by grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Use the best model obtained from grid search
best_xgb_model = grid_search.best_estimator_

# Predict star ratings using the best model
y_pred_xgb_tuned = best_xgb_model.predict(X_test)

# Calculate RMSE for the tuned XGBoost model
rmse_xgb_tuned = mean_squared_error(y_test, y_pred_xgb_tuned, squared=False)
print("Root Mean Squared Error (Tuned XGBoost):", rmse_xgb_tuned)

# Evaluate accuracy on the training dataset using the tuned model
y_train_pred_xgb_tuned = best_xgb_model.predict(X_train)
train_rmse_xgb_tuned = mean_squared_error(y_train, y_train_pred_xgb_tuned, squared=False)
print("Training Root Mean Squared Error (Tuned XGBoost):", train_rmse_xgb_tuned)


Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
Root Mean Squared Error (Tuned XGBoost): 1.1957062574547777
Training Root Mean Squared Error (Tuned XGBoost): 0.8854691898029619


# Gradient Boosting Regressor

In [116]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Initialize and fit Gradient Boosting regressor model
gb_model = GradientBoostingRegressor()
gb_model.fit(X_train, y_train)

# Predict star ratings
y_pred_gb = gb_model.predict(X_test)

# Calculate RMSE for Gradient Boosting model
rmse_gb = mean_squared_error(y_test, y_pred_gb, squared=False)
print("Root Mean Squared Error (Gradient Boosting):", rmse_gb)

# Evaluate accuracy on the training dataset
y_train_pred_gb = gb_model.predict(X_train)
train_rmse_gb = mean_squared_error(y_train, y_train_pred_gb, squared=False)
print("Training Root Mean Squared Error (Gradient Boosting):", train_rmse_gb)


Root Mean Squared Error (Gradient Boosting): 1.1929651199206288
Training Root Mean Squared Error (Gradient Boosting): 1.0507663457855112


# TASK 3

# Five businesses similar to 'Spring Chinese Restaurant' in Philadelphia of type Restaurants

In [45]:
import pandas as pd

# List to store restaurant information
restaurant_data = []

# Iterate through the business IDs and store restaurant information
for business_id in business_embeddings.keys():
    restaurant_info = df_businesses[df_businesses['business_id'] == business_id]
    if not restaurant_info.empty:
        categories = restaurant_info['categories'].values[0]
        if isinstance(categories, str) and 'Restaurants' in categories:
            restaurant_name = restaurant_info['name'].values[0]
            city = restaurant_info['city'].values[0]
            restaurant_data.append([business_id, restaurant_name, city])

# Create a DataFrame from the restaurant data
restaurant_df = pd.DataFrame(restaurant_data, columns=["Business ID", "Restaurant Name", "City"])

restaurant_df.head()


,Business ID,Restaurant Name,City
0,--ZVrH2X2QXBFdCilbirsw,Chris's Sandwich Shop,Ardmore
1,-1MhPXk1FglglUAmuPLIGg,Red Garter Saloon,Tucson
2,-2Axhv9AZ_n7qjQefECpVw,Greek Sotir Grill,Hermitage
3,-2ke_JDOpgTZWqirMFjZcw,Nite Owls Saloon & Grill,Beech Grove
4,-3AooxIkg38UyUdlz5oXdw,Chase Restaurant,Santa Barbara


In [46]:
# Get the count of items in business_embeddings
num_items = len(business_embeddings)

# Print the count
print("Number of items in business_embeddings:", num_items)


Number of items in business_embeddings: 9406


In [47]:
# Get the count of items in business_embeddings
num_items = len(restaurant_df)

# Print the count
print("Number of items in business_embeddings matching df_businesses for restaurants:", num_items)

Number of items in business_embeddings matching df_businesses for restaurants: 4022


In [48]:
restaurant_df.head(10)

,Business ID,Restaurant Name,City
0,--ZVrH2X2QXBFdCilbirsw,Chris's Sandwich Shop,Ardmore
1,-1MhPXk1FglglUAmuPLIGg,Red Garter Saloon,Tucson
2,-2Axhv9AZ_n7qjQefECpVw,Greek Sotir Grill,Hermitage
3,-2ke_JDOpgTZWqirMFjZcw,Nite Owls Saloon & Grill,Beech Grove
4,-3AooxIkg38UyUdlz5oXdw,Chase Restaurant,Santa Barbara
5,-49YlJ2GGzgmQ71FX9gGyg,Pineapple Asian Restaurant,Berlin Township
6,-4E0hSCldRJZLI-1cT38Sw,Smoothie King,Smyrna
7,-6OjnX3ZdDOhHxWR60wysg,BlackFinn American Grille,Saint Louis
8,-6TttC-1-rf5gC0QHWrs_w,Goody's Cafe,Terrytown
9,-7GDqSUaXrpC8Ql7nDBxWA,B2 Bluefin,Bala Cynwyd


In [49]:
# Count businesses in each city
city_counts = restaurant_df['City'].value_counts()

# Display the count of businesses in each city
print("Count of restaurants in each city:")
city_counts.head(15)

Count of restaurants in each city:


Philadelphia        473
Tampa               222
Indianapolis        210
Nashville           192
New Orleans         192
Tucson              183
Edmonton            165
Saint Louis         150
Reno                104
Santa Barbara        58
Boise                56
Saint Petersburg     44
Clearwater           41
Wilmington           40
Franklin             39
Name: City, dtype: int64

In [50]:
# Filter restaurants from Philadelphia
philadelphia_restaurants = restaurant_df[restaurant_df['City'] == 'Philadelphia']

# Display the list of restaurants from Philadelphia
print(philadelphia_restaurants.head(20))


                Business ID                    Restaurant Name          City
12   -ATiAtTikuGuqvaW2O6tNA                           Marathon  Philadelphia
16   -CRjqZ4GxZ6lIa_qMj7-yw          Spring Chinese Restaurant  Philadelphia
28   -OIUunijjcq_ZzyyQhPPFQ                    Zorba's Taverna  Philadelphia
29   -Or44IdY51Ukd618kikmtA                     Federal Donuts  Philadelphia
35   -V0vIgo6196MDn_x3ZaYmA                   La Creperie Cafe  Philadelphia
77   04UD14gamNjLY0IDYVhHJg                           Dmitri's  Philadelphia
83   0CEUpFtaKhxbmrbSdP9MUQ                             Acadia  Philadelphia
91   0JoB3ThhI-DS78-1Ks1EUQ                       Hibachi 2 Go  Philadelphia
97   0a8t_Th0jtWUMmtq0q2dqA                           DK Sushi  Philadelphia
100  0dfspevYKsugYiU1UVl94Q                Brewerytown Taproom  Philadelphia
106  0mijbNf-7wiKJ9xto1pAyg                        Spice Finch  Philadelphia
124  0zWAsCBzvuCI_Ydx7yH1Jw                          Shing Kee  Philadelphia

In [51]:
import pandas as pd

# Get popular categories
popular_categories = df_businesses['categories'].str.split(', ', expand=True).stack().value_counts()

print("Popular categories in df_businesses:")
print(popular_categories.head(20))


Popular categories in df_businesses:
Restaurants                  34749
Food                         18517
Shopping                     16277
Home Services                 9622
Beauty & Spas                 9504
Nightlife                     8124
Health & Medical              7835
Local Services                7470
Bars                          7327
Automotive                    7131
Event Planning & Services     6605
Sandwiches                    5568
American (Traditional)        5368
Active Life                   5135
Pizza                         4697
Coffee & Tea                  4435
Fast Food                     4233
Breakfast & Brunch            4113
American (New)                4073
Fashion                       3875
dtype: int64


In [55]:
import numpy as np

# Function to calculate cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

# Function to find similar businesses
def find_similar_businesses(target_business_name, business_embeddings, businesses_table, city, business_type, num_similar=5):
    similar_businesses = []
    
    # Filter businesses in the specified city and business type
    city_businesses = businesses_table[(businesses_table['city'] == city) & 
                                       (businesses_table['categories'].str.contains(business_type))]
    
    # Check if the target business name exists in the dataframe
    if target_business_name not in city_businesses['name'].values:
        print("Target business '{}' not found in the specified city.".format(target_business_name))
        return similar_businesses
    
    # Get the business ID for the target business name
    target_business_id = city_businesses[city_businesses['name'] == target_business_name]['business_id'].values[0]
    
    # Check if the target business ID exists in the embeddings
    if target_business_id not in business_embeddings:
        print("Target business '{}' not found in embeddings.".format(target_business_name))
        return similar_businesses
    
    # Get the embedding for the target business
    target_embedding = business_embeddings[target_business_id]
    
    # Compare each business's embedding with the target embedding
    for idx, row in city_businesses.iterrows():
        business_id = row['business_id']
        business_embedding = business_embeddings.get(business_id)
        
        if business_embedding is not None:
            similarity = cosine_similarity(target_embedding, business_embedding)
            similar_businesses.append((row['name'], row['stars'], row['categories'], similarity))
    
    # Sort the businesses by similarity and select the top num_similar
    similar_businesses.sort(key=lambda x: x[3], reverse=True)
    similar_businesses = similar_businesses[:num_similar]
    
    return similar_businesses

# Specify the target business, city, and business type
target_business_name = 'Spring Chinese Restaurant'
city = 'Philadelphia'
business_type = 'Restaurants'

# Print the target business
print("Target business in {} of type {}: {}".format(city, business_type, target_business_name))

# Find similar businesses
similar_businesses = find_similar_businesses(target_business_name, business_embeddings, df_businesses, city, business_type, num_similar=6)

# Display the similar businesses
print("\nFive businesses similar to '{}' in {} of type {}:".format(target_business_name, city, business_type))
for idx, (name, stars, categories, similarity) in enumerate(similar_businesses, 1):
    print("{}. Name: {}, Stars: {}, Categories: {}, Similarity: {:.4f}".format(idx, name, stars, categories, similarity))


Target business in Philadelphia of type Restaurants: Spring Chinese Restaurant

Five businesses similar to 'Spring Chinese Restaurant' in Philadelphia of type Restaurants:
1. Name: Spring Chinese Restaurant, Stars: 3.5, Categories: Chinese, Restaurants, Similarity: 1.0000
2. Name: Golden Chopstick Chinese Restaurant, Stars: 3.0, Categories: Restaurants, Chinese, Similarity: 0.9963
3. Name: Jamaican Jerk Hut, Stars: 2.5, Categories: Caribbean, Restaurants, Similarity: 0.9944
4. Name: Capital Beer, Stars: 3.5, Categories: Chinese, Restaurants, Sushi Bars, Similarity: 0.9937
5. Name: Ruby Garden Restaurant, Stars: 3.0, Categories: Dim Sum, Diners, Seafood, Restaurants, American (Traditional), Chinese, Cantonese, Similarity: 0.9936
6. Name: Panasian Buffet, Stars: 3.5, Categories: Pan Asian, Japanese, Buffets, Restaurants, American (Traditional), Chinese, Seafood, Similarity: 0.9935


# Five businesses similar to 'Queen Sheba' in Philadelphia of type Bars

# Let's use a different Similarity Measure

In [58]:
import pandas as pd

# List to store restaurant information
bars_data = []

# Iterate through the business IDs and store restaurant information
for business_id in business_embeddings.keys():
    bars_info = df_businesses[df_businesses['business_id'] == business_id]
    if not bars_info.empty:
        categories = bars_info['categories'].values[0]
        if isinstance(categories, str) and 'Bars' in categories:
            bar_name = bars_info['name'].values[0]
            city = bars_info['city'].values[0]
            bars_data.append([business_id, bar_name, city])

# Create a DataFrame from the restaurant data
bars_df = pd.DataFrame(bars_data, columns=["Business ID", "Bars Name", "City"])

bars_df.head(15)


,Business ID,Bars Name,City
0,-1MhPXk1FglglUAmuPLIGg,Red Garter Saloon,Tucson
1,-2ke_JDOpgTZWqirMFjZcw,Nite Owls Saloon & Grill,Beech Grove
2,-3AooxIkg38UyUdlz5oXdw,Chase Restaurant,Santa Barbara
3,-4E0hSCldRJZLI-1cT38Sw,Smoothie King,Smyrna
4,-7GDqSUaXrpC8Ql7nDBxWA,B2 Bluefin,Bala Cynwyd
5,-7GjicSH_rM8JeZGCXGcUg,Double Decker,Tampa
6,-ATiAtTikuGuqvaW2O6tNA,Marathon,Philadelphia
7,-AhpS4MUSdYWg8SwZpV6eA,Tokyo Express,Edmonton
8,-J-Ue2JzFqYpu2XdTboMzw,The Office,Saint Louis
9,-TCa3KBib07_1ko9L2Z0fQ,Yeoman's Cask and Lion,St. Petersburg


In [59]:
bars_df.tail(15)

,Business ID,Bars Name,City
1053,zSEFzAbxnO44w8ACQ_XkAA,Akira Hibachi & Sushi Bar,Riverview
1054,zSsdeiHfEnv_EVv8IVqcWg,eegee's,Tucson
1055,zTdUzhwCUDglY6bHJUEuzg,Kabuki Sushi and Grill,Edmonton
1056,zTp2lpm3z5tVk62rA-pCdQ,Rooftop Garden,Indianapolis
1057,zXgSqLVumgxM_e5OgWfUTw,Press Start,Reno
1058,zYy9lS8HlpbCHsGPmORTfA,Twisted Roots,St. Louis
1059,zbrIMldF_O1ZQ0vpUaaa8A,Bluewater Grill - Santa Barbara,Santa Barbara
1060,zedtiorJSVxjpqMKdyp9TA,Bully's Sports Bar & Grill,Reno
1061,zjQDk4tZyhEroyqtkgvx0g,The Cobblestone Eatery and Drinkery,Lebanon
1062,zmpRwOqxaajeh2YvAKxLaQ,Morton's The Steakhouse,King Of Prussia


In [61]:
# Count businesses in each city
city_counts = bars_df['City'].value_counts()

# Display the count of businesses in each city
print("Count of restaurants in each city:")
city_counts.head(15)

Count of restaurants in each city:


Philadelphia        139
New Orleans          73
Tampa                71
Nashville            69
Saint Louis          53
Tucson               47
Reno                 45
Indianapolis         45
Edmonton             41
Santa Barbara        24
Saint Petersburg     18
St. Petersburg       13
Boise                13
St. Louis            12
Clearwater           11
Name: City, dtype: int64

In [60]:
import numpy as np

# Function to calculate Jaccard similarity between two vectors
def jaccard_similarity(vec1, vec2):
    intersection = np.sum(np.minimum(vec1, vec2))  # Count the number of overlapping elements
    union = np.sum(np.maximum(vec1, vec2))  # Count the total number of elements
    return intersection / union

# Function to find similar businesses using Jaccard similarity
def find_similar_businesses_jaccard(target_business_name, business_embeddings, businesses_table, city, business_type, num_similar=5):
    similar_businesses = []
    
    # Filter businesses in the specified city and business type
    city_businesses = businesses_table[(businesses_table['city'] == city) & 
                                       (businesses_table['categories'].str.contains(business_type))]
    
    # Check if the target business name exists in the dataframe
    if target_business_name not in city_businesses['name'].values:
        print("Target business '{}' not found in the specified city.".format(target_business_name))
        return similar_businesses
    
    # Get the business ID for the target business name
    target_business_id = city_businesses[city_businesses['name'] == target_business_name]['business_id'].values[0]
    
    # Check if the target business ID exists in the embeddings
    if target_business_id not in business_embeddings:
        print("Target business '{}' not found in embeddings.".format(target_business_name))
        return similar_businesses
    
    # Get the embedding for the target business
    target_embedding = business_embeddings[target_business_id]
    
    # Compare each business's embedding with the target embedding
    for idx, row in city_businesses.iterrows():
        business_id = row['business_id']
        business_embedding = business_embeddings.get(business_id)
        
        if business_embedding is not None:
            similarity = jaccard_similarity(target_embedding, business_embedding)
            similar_businesses.append((row['name'], row['stars'], row['categories'], similarity))
    
    # Sort the businesses by similarity and select the top num_similar
    similar_businesses.sort(key=lambda x: x[3], reverse=True)
    similar_businesses = similar_businesses[:num_similar]
    
    return similar_businesses

# Specify the target business, city, and business type
target_business_name = 'Queen Sheba'
city = 'Philadelphia'
business_type = 'Bars'

# Print the target business
print("Target business in {} of type {}: {}".format(city, business_type, target_business_name))

# Find similar businesses using Jaccard similarity
similar_businesses_jaccard = find_similar_businesses_jaccard(target_business_name, business_embeddings, df_businesses, city, business_type, num_similar=6)

# Display the similar businesses
print("\nFive businesses similar to '{}' in {} of type {} using Jaccard similarity:".format(target_business_name, city, business_type))
for idx, (name, stars, categories, similarity) in enumerate(similar_businesses_jaccard, 1):
    print("{}. Name: {}, Stars: {}, Categories: {}, Similarity: {:.4f}".format(idx, name, stars, categories, similarity))


Target business in Philadelphia of type Bars: Queen Sheba

Five businesses similar to 'Queen Sheba' in Philadelphia of type Bars using Jaccard similarity:
1. Name: Queen Sheba, Stars: 4.0, Categories: Nightlife, Restaurants, Sports Bars, Bars, Ethiopian, Similarity: 1.0000
2. Name: Smiley's Cafe, Stars: 4.5, Categories: Juice Bars & Smoothies, Food, Mediterranean, Greek, Restaurants, Sandwiches, Similarity: 0.6124
3. Name: Prohibition Taproom, Stars: 4.0, Categories: American (Traditional), Bars, Nightlife, Restaurants, Beer Bar, Gastropubs, Breakfast & Brunch, Tapas/Small Plates, Similarity: 0.5704
4. Name: Pub & Kitchen, Stars: 3.5, Categories: Restaurants, Pubs, Food, Bars, American (New), Nightlife, Similarity: 0.5684
5. Name: Mac's Tavern, Stars: 3.5, Categories: Nightlife, Restaurants, American (New), Bars, Pubs, Similarity: 0.5617
6. Name: The Black Sheep Pub & Restaurant, Stars: 3.5, Categories: Bars, Pubs, Irish Pub, Nightlife, Sports Bars, Restaurants, American (New), Similar

# Five businesses similar to ''Soho Wellness & Med Spa'' in Tampa of types ['Health & Medical', 'Beauty & Spas']

In [107]:
import pandas as pd

# List to store business information
business_data = []

# Iterate through the business IDs and store business information
for business_id in business_embeddings.keys():
    business_info = df_businesses[df_businesses['business_id'] == business_id]
    if not business_info.empty:
        categories = business_info['categories'].values[0]
        if isinstance(categories, str) and ('Health & Medical' in categories and 'Beauty & Spas' in categories):
            business_name = business_info['name'].values[0]
            city = business_info['city'].values[0]
            business_data.append([business_id, business_name, city])

# Create a DataFrame from the business data
business_df = pd.DataFrame(business_data, columns=["Business ID", "Business Name", "City"])

business_df.head(15)


,Business ID,Business Name,City
0,-uq7NWYxWNG6Pemh1Y7O4Q,The Joint Chiropractic,Goleta
1,0JbAdxxS1aZZxCjucbTuAQ,Face Defined,Boise
2,0Zvrh9rCrThh1ZwGaTbDjA,Sound Stone Massage Spa,St. Louis
3,0wdwwtbTGTF5SIJLmGRn3g,Qubo Salon & Spa,Edmonton
4,1YFkAjr87WDMEFre4Kp9mw,Advanced ENT & Allergy,Voorhees Township
5,1aOjfsDJOZMCKOi0xro3Xw,Feel Better Reflexology Center,Philadelphia
6,3uw1vR4E5TCrxEmDW12_Mw,The Balance Within,Willow Grove
7,4-MogH7DSctJCHmAwUoUug,Lakeside Massage Spa,Metairie
8,4LtDvcwOauXA1tIwGU5uyw,Soho Wellness & Med Spa,Tampa
9,4eJT2BdTk57LuyEXMPzYGg,Massage Addict | Edmonton Northwest,Edmonton


In [108]:
import numpy as np

# Function to calculate cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

# Function to find similar businesses using cosine similarity
def find_similar_businesses_cosine(target_business_name, business_embeddings, businesses_table, city, business_types, num_similar=5):
    similar_businesses = []
    
    # Filter businesses in the specified city and business types
    city_businesses = businesses_table[(businesses_table['city'] == city) & 
                                       (businesses_table['categories'].str.contains('|'.join(business_types)))]
    
    # Check if the target business name exists in the dataframe
    if target_business_name not in city_businesses['name'].values:
        print("Target business '{}' not found in the specified city.".format(target_business_name))
        return similar_businesses
    
    # Get the business ID for the target business name
    target_business_id = city_businesses[city_businesses['name'] == target_business_name]['business_id'].values[0]
    
    # Check if the target business ID exists in the embeddings
    if target_business_id not in business_embeddings:
        print("Target business '{}' not found in embeddings.".format(target_business_name))
        return similar_businesses
    
    # Get the embedding for the target business
    target_embedding = business_embeddings[target_business_id]
    
    # Compare each business's embedding with the target embedding using cosine similarity
    for idx, row in city_businesses.iterrows():
        business_id = row['business_id']
        business_embedding = business_embeddings.get(business_id)
        
        if business_embedding is not None:
            similarity = cosine_similarity(target_embedding, business_embedding)
            # Combine business types for the categories
            combined_categories = ', '.join(business_types)
            similar_businesses.append((row['name'], row['stars'], combined_categories, similarity))
    
    # Sort the businesses by similarity and select the top num_similar
    similar_businesses.sort(key=lambda x: x[3], reverse=True)
    similar_businesses = similar_businesses[:num_similar]
    
    return similar_businesses

# Specify the target business, city, and business types
target_business_name = 'Soho Wellness & Med Spa'
city = 'Tampa'
business_types = ['Health & Medical', 'Beauty & Spas']

# Print the target business
print("Target business in {} of types {}: {}".format(city, business_types, target_business_name))

# Find similar businesses using cosine similarity
similar_businesses_cosine = find_similar_businesses_cosine(target_business_name, business_embeddings, df_businesses, city, business_types, num_similar=6)

# Display the similar businesses using cosine similarity
print("\nFive businesses similar to '{}' in {} of types {} :".format(target_business_name, city, business_types))
for idx, (name, stars, categories, similarity) in enumerate(similar_businesses_cosine, 1):
    print("{}. Name: {}, Stars: {}, Categories: {}, Similarity: {:.4f}".format(idx, name, stars, categories, similarity))


Target business in Tampa of types ['Health & Medical', 'Beauty & Spas']: Soho Wellness & Med Spa

Five businesses similar to 'Soho Wellness & Med Spa' in Tampa of types ['Health & Medical', 'Beauty & Spas'] :
1. Name: Soho Wellness & Med Spa, Stars: 4.5, Categories: Health & Medical, Beauty & Spas, Similarity: 1.0000
2. Name: Massage Studio, Stars: 4.5, Categories: Health & Medical, Beauty & Spas, Similarity: 0.9931
3. Name: Massage Studio, Stars: 4.5, Categories: Health & Medical, Beauty & Spas, Similarity: 0.9875
4. Name: Love Nail & Spa, Stars: 4.0, Categories: Health & Medical, Beauty & Spas, Similarity: 0.9875
5. Name: Lecada Medical Artistry, Stars: 4.5, Categories: Health & Medical, Beauty & Spas, Similarity: 0.9871
6. Name: Manhattan Nails Salon, Stars: 4.0, Categories: Health & Medical, Beauty & Spas, Similarity: 0.9860


# Five businesses similar to 'Safelite AutoGlass' in Tampa of types ['Automotive', 'Windshield Installation & Repair']

In [100]:
import pandas as pd

# List to store business information
business_data = []

# Iterate through the business IDs and store business information
for business_id in business_embeddings.keys():
    business_info = df_businesses[df_businesses['business_id'] == business_id]
    if not business_info.empty:
        categories = business_info['categories'].values[0]
        if isinstance(categories, str) and ('Automotive' in categories and 'Windshield Installation & Repair' in categories):
            business_name = business_info['name'].values[0]
            city = business_info['city'].values[0]
            business_data.append([business_id, business_name, city])

# Create a DataFrame from the business data
business_df = pd.DataFrame(business_data, columns=["Business ID", "Business Name", "City"])


In [101]:
business_df.head(25)

,Business ID,Business Name,City
0,1wvgDi3NJe_qFPPehYIw9g,Shall-Martin Garage,Newtown Square
1,6NBovZlRhbFD2KqwTVVDgA,Vic's Auto Glass,New Orleans
2,8Bnn5sFX3KJ2CbEG9e16aA,Safelite AutoGlass,Fairless Hills
3,8zo6B_ZylluQcC3Pi8FP3w,Safelite AutoGlass,Tucson
4,9BVa8bTw9FDemqkQUQXDWg,Glass Doctor of Southwest Idaho,Meridian
5,Dt26KHsIfgnS1q52HKgIcw,Glass Doctor of Greater St. Louis,St. Louis
6,FpubPVZtYda5mXzGMNCaDQ,Abra Auto Body Repair,Boise
7,GgGfhKUGusCbP-rvjHiS3A,Safelite AutoGlass,Tampa
8,HI4_NqeHMJOnMoVFNxaFyA,1-Hour Auto Glass,Tucson
9,LzujgmlZDn3DtFbtbwmisA,Valley Glass - Boise,Boise


In [105]:
import numpy as np

# Function to calculate cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

# Function to find similar businesses using cosine similarity
def find_similar_businesses_cosine(target_business_name, business_embeddings, businesses_table, city, business_types, num_similar=5):
    similar_businesses = []
    
    # Filter businesses in the specified city and business types
    city_businesses = businesses_table[(businesses_table['city'] == city) & 
                                       (businesses_table['categories'].str.contains('|'.join(business_types)))]
    
    # Check if the target business name exists in the dataframe
    if target_business_name not in city_businesses['name'].values:
        print("Target business '{}' not found in the specified city.".format(target_business_name))
        return similar_businesses
    
    # Get the business ID for the target business name
    target_business_id = city_businesses[city_businesses['name'] == target_business_name]['business_id'].values[0]
    
    # Check if the target business ID exists in the embeddings
    if target_business_id not in business_embeddings:
        print("Target business '{}' not found in embeddings.".format(target_business_name))
        return similar_businesses
    
    # Get the embedding for the target business
    target_embedding = business_embeddings[target_business_id]
    
    # Compare each business's embedding with the target embedding using cosine similarity
    for idx, row in city_businesses.iterrows():
        business_id = row['business_id']
        business_embedding = business_embeddings.get(business_id)
        
        if business_embedding is not None:
            similarity = cosine_similarity(target_embedding, business_embedding)
            # Combine business types for the categories
            combined_categories = ', '.join(business_types)
            similar_businesses.append((row['name'], row['stars'], combined_categories, similarity))
    
    # Sort the businesses by similarity and select the top num_similar
    similar_businesses.sort(key=lambda x: x[3], reverse=True)
    similar_businesses = similar_businesses[:num_similar]
    
    return similar_businesses

# Specify the target business, city, and business types
target_business_name = 'Safelite AutoGlass'
city = 'Tampa'
business_types = ['Automotive','Windshield Installation & Repair']

# Print the target business
print("Target business in {} of types {}: {}".format(city, business_types, target_business_name))

# Find similar businesses using cosine similarity
similar_businesses_cosine = find_similar_businesses_cosine(target_business_name, business_embeddings, df_businesses, city, business_types, num_similar=6)

# Display the similar businesses using cosine similarity
print("\nFive businesses similar to '{}' in {} of types {} :".format(target_business_name, city, business_types))
for idx, (name, stars, categories, similarity) in enumerate(similar_businesses_cosine, 1):
    print("{}. Name: {}, Stars: {}, Categories: {}, Similarity: {:.4f}".format(idx, name, stars, categories, similarity))


Target business in Tampa of types ['Automotive', 'Windshield Installation & Repair']: Safelite AutoGlass

Five businesses similar to 'Safelite AutoGlass' in Tampa of types ['Automotive', 'Windshield Installation & Repair'] :
1. Name: Safelite AutoGlass, Stars: 3.0, Categories: Automotive, Windshield Installation & Repair, Similarity: 1.0000
2. Name: Courtesy Hyundai Tampa, Stars: 3.0, Categories: Automotive, Windshield Installation & Repair, Similarity: 0.9924
3. Name: Audio Itch, Stars: 3.5, Categories: Automotive, Windshield Installation & Repair, Similarity: 0.9892
4. Name: Sears Auto Center, Stars: 2.0, Categories: Automotive, Windshield Installation & Repair, Similarity: 0.9884
5. Name: Wheel Tec, Stars: 4.0, Categories: Automotive, Windshield Installation & Repair, Similarity: 0.9876
6. Name: Ed Morse Cadillac Tampa, Stars: 3.5, Categories: Automotive, Windshield Installation & Repair, Similarity: 0.9873


# Five businesses similar to 'Sociale Italian Tapas & Pizza Bar' in Tampa of types ['Pizza', 'Bars']

In [112]:
import pandas as pd

# List to store business information
business_data = []

# Iterate through the business IDs and store business information
for business_id in business_embeddings.keys():
    business_info = df_businesses[df_businesses['business_id'] == business_id]
    if not business_info.empty:
        categories = business_info['categories'].values[0]
        if isinstance(categories, str) and ('Pizza' in categories and 'Bars' in categories):
            business_name = business_info['name'].values[0]
            city = business_info['city'].values[0]
            business_data.append([business_id, business_name, city])

# Create a DataFrame from the business data
business_df = pd.DataFrame(business_data, columns=["Business ID", "Business Name", "City"])


In [113]:
business_df.head(25)

,Business ID,Business Name,City
0,-sSGKR1V1oBm7BJicIXozg,Barra Rossa Ristorante & Wine Bar - Downington,Downingtown
1,05471AhNLv1NxKoaVs4gYw,Sam's Brick Oven Pizza,Ardmore
2,10JlCiyViBpUqRf58XlbZg,Pasta Vino,Berlin
3,1IjHo5kaSdFxJkJotU4O7A,Sociale Italian Tapas & Pizza Bar,Tampa
4,1r2bAE5TQpUO-zec9-cvxg,Red Eagle Tavern,Cherry Hill
5,2SYEGA2fZLdzZQ9lmIfM5Q,Urban Village Brewing Co,Philadelphia
6,2W0P6hXCbE6EgP5R6y9eBw,Pearl Street Pizzeria & Pub,Indianapolis
7,2rQFgavS-942RapZTBSBTw,Gullifty's Restaurant,Bryn Mawr
8,2wwhMKlfpOd0dpk_AQdt8w,Leonardo Trattoria,New Orleans
9,5PDNXMquh1C9E089G1RY2g,Ob's Pizza Pub & Eatery,Indianapolis


In [114]:
import numpy as np

# Function to calculate cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

# Function to find similar businesses using cosine similarity
def find_similar_businesses_cosine(target_business_name, business_embeddings, businesses_table, city, business_types, num_similar=5):
    similar_businesses = []
    
    # Filter businesses in the specified city and business types
    city_businesses = businesses_table[(businesses_table['city'] == city) & 
                                       (businesses_table['categories'].str.contains('|'.join(business_types)))]
    
    # Check if the target business name exists in the dataframe
    if target_business_name not in city_businesses['name'].values:
        print("Target business '{}' not found in the specified city.".format(target_business_name))
        return similar_businesses
    
    # Get the business ID for the target business name
    target_business_id = city_businesses[city_businesses['name'] == target_business_name]['business_id'].values[0]
    
    # Check if the target business ID exists in the embeddings
    if target_business_id not in business_embeddings:
        print("Target business '{}' not found in embeddings.".format(target_business_name))
        return similar_businesses
    
    # Get the embedding for the target business
    target_embedding = business_embeddings[target_business_id]
    
    # Compare each business's embedding with the target embedding using cosine similarity
    for idx, row in city_businesses.iterrows():
        business_id = row['business_id']
        business_embedding = business_embeddings.get(business_id)
        
        if business_embedding is not None:
            similarity = cosine_similarity(target_embedding, business_embedding)
            # Combine business types for the categories
            combined_categories = ', '.join(business_types)
            similar_businesses.append((row['name'], row['stars'], combined_categories, similarity))
    
    # Sort the businesses by similarity and select the top num_similar
    similar_businesses.sort(key=lambda x: x[3], reverse=True)
    similar_businesses = similar_businesses[:num_similar]
    
    return similar_businesses

# Specify the target business, city, and business types
target_business_name = 'Sociale Italian Tapas & Pizza Bar'
city = 'Tampa'
business_types = ['Pizza','Bars']

# Print the target business
print("Target business in {} of types {}: {}".format(city, business_types, target_business_name))

# Find similar businesses using cosine similarity
similar_businesses_cosine = find_similar_businesses_cosine(target_business_name, business_embeddings, df_businesses, city, business_types, num_similar=6)

# Display the similar businesses using cosine similarity
print("\nFive businesses similar to '{}' in {} of types {} :".format(target_business_name, city, business_types))
for idx, (name, stars, categories, similarity) in enumerate(similar_businesses_cosine, 1):
    print("{}. Name: {}, Stars: {}, Categories: {}, Similarity: {:.4f}".format(idx, name, stars, categories, similarity))


Target business in Tampa of types ['Pizza', 'Bars']: Sociale Italian Tapas & Pizza Bar

Five businesses similar to 'Sociale Italian Tapas & Pizza Bar' in Tampa of types ['Pizza', 'Bars'] :
1. Name: Sociale Italian Tapas & Pizza Bar, Stars: 4.0, Categories: Pizza, Bars, Similarity: 1.0000
2. Name: The C House, Stars: 4.0, Categories: Pizza, Bars, Similarity: 0.9959
3. Name: Timpano Hyde Park, Stars: 3.5, Categories: Pizza, Bars, Similarity: 0.9952
4. Name: American Social, Stars: 3.5, Categories: Pizza, Bars, Similarity: 0.9951
5. Name: Precinct Pizza, Stars: 3.5, Categories: Pizza, Bars, Similarity: 0.9950
6. Name: Sake House, Stars: 3.0, Categories: Pizza, Bars, Similarity: 0.9950
